## Fitting a NN

This tutorial is clearly a WIP. It's here to demonstrate the fitting works. To see more details on the nn part, go to nn_dev.jl for now.

### Defining a neuron
A neuron is just a container for weights, a bias, and an activation function. It takes a number of inputs, combines them, and produces a scalar output. The number of weights is determined by the number of inputs (features) to that neuron. The activation function allows for more complex mapping of inputs to the output (and so that layers of neurons don't reduce to a single linear combination). The code defining a neuron is here:

```julia
    mutable struct Neuron{T<:AbstractFloat,F<:Union{Function,Nothing}}
        w::Vector{Value{T}}
        b::Value{T}
        activation::F
    end
```
We can define a function to make variables of this type "callable": `n(X)`

```julia
    function (n::Neuron)(x)
        if length(x) != length(n.w)
            error("In calling n(x), expected $(length(n.w)) inputs to neuron, got $(length(x)):\n\tx = $(x)")
        end
        raw = sum(n.w .* x)+ n.b
        isnothing(n.activation) && return raw 
        return n.activation(raw)
    end
```
This just computes the inner product of X and the weights, adds b, and then runs the result through an activation function.

In [2]:
using Micrograd

X,y = getmoons()

m = mlp(2, [16, 16, 1])


fit(m,X,y)

Step: 0	Loss: 0.92	Accuracy: 0.5


Step: 1	Loss: 0.31	Accuracy: 0.89


Step: 2	Loss: 1.6	Accuracy: 0.5


Step: 3	Loss: 5.7	Accuracy: 0.5


Step: 4	Loss: 0.81	Accuracy: 0.5


...

Step: 95	Loss: 0.015	Accuracy: 1.0


Step: 96	Loss: 0.015	Accuracy: 1.0


Step: 97	Loss: 0.015	Accuracy: 1.0


Step: 98	Loss: 0.015	Accuracy: 1.0


Step: 99	Loss: 0.015	Accuracy: 1.0


In [6]:
y_fit = m.(X)
cat(y,sign.(getfield.(y_fit,:data)),dims=2)

100×2 Matrix{Float64}:
 -1.0  -1.0
 -1.0  -1.0
  1.0   1.0
 -1.0  -1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
 -1.0  -1.0
  ⋮    
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
 -1.0  -1.0
  1.0   1.0
  1.0   1.0
 -1.0  -1.0